# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [3]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [5]:
import syft as sy

In [6]:
hook = sy.TorchHook(th)

In [7]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [8]:
bob = sy.VirtualWorker(hook, id="bob")

In [10]:
bob._objects

{}

In [11]:
x = th.tensor([1,2,3,4,5])

In [12]:
x = x.send(bob)

In [13]:
bob._objects

{76340360880: tensor([1, 2, 3, 4, 5])}

In [14]:
x.location

<VirtualWorker id:bob #objects:1>

In [15]:
x.id_at_location

76340360880

In [16]:
x.id

4519696570

In [17]:
x.owner

<VirtualWorker id:me #objects:0>

In [18]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [19]:
x

(Wrapper)>[PointerTensor | me:4519696570 -> bob:76340360880]

In [20]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [21]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [ ]:
# try this project here!

In [22]:
alice = sy.VirtualWorker(hook, id='alice')

In [25]:
x = th.tensor([1, 2, 3, 4])

In [26]:
x_ptr = x.send(bob, alice)

In [28]:
x_ptr.get()

[tensor([1, 2, 3, 4]), tensor([1, 2, 3, 4])]

In [32]:
x = th.tensor([1, 2, 3, 4, 5]).send(bob, alice)

In [33]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

# Lesson: Introducing Remote Arithmetic

In [34]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [35]:
x

(Wrapper)>[PointerTensor | me:34283416228 -> bob:70949416198]

In [36]:
y

(Wrapper)>[PointerTensor | me:81603846854 -> bob:61842475145]

In [37]:
z = x + y

In [38]:
z

(Wrapper)>[PointerTensor | me:87761636475 -> bob:43562771561]

In [39]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [40]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:21601341372 -> bob:25041042452]

In [41]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [42]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [43]:
z = (x + y).sum()

In [44]:
z.backward()

(Wrapper)>[PointerTensor | me:66868872952 -> bob:73773656645]

In [45]:
x = x.get()

In [46]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [40]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [ ]:
# try this project here!

In [74]:
input = th.tensor([[1., 1], [0, 1, ], [1, 0], [0, 0]], requires_grad=True).send(bob)
target = th.tensor([[1.], [1], [0], [0]], requires_grad=True).send(bob)

In [75]:
input

(Wrapper)>[PointerTensor | me:13427005585 -> bob:66922134771]

In [76]:
target

(Wrapper)>[PointerTensor | me:67103161779 -> bob:38049005565]

In [77]:
weights = th.tensor([[0.], [0.]], requires_grad=True).send(bob)

In [78]:
for i in range(10):
    pred = input.mm(weights)

    loss = ((pred - target)**2).sum()

    loss.backward()

    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0

    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


# Lesson: Garbage Collection and Common Errors


In [79]:
bob = bob.clear_objects()

In [80]:
bob._objects

{}

In [81]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [82]:
bob._objects

{33589677577: tensor([1, 2, 3, 4, 5])}

In [83]:
del x

In [84]:
bob._objects

{}

In [85]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [88]:
x.child.garbage_collect_data

False

In [89]:
bob._objects

{71226330779: tensor([1, 2, 3, 4, 5])}

In [90]:
x = "asdf"

In [91]:
bob._objects

{71226330779: tensor([1, 2, 3, 4, 5])}

In [92]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [93]:
x

(Wrapper)>[PointerTensor | me:26500260348 -> bob:23985120382]

In [94]:
bob._objects

{71226330779: tensor([1, 2, 3, 4, 5]), 23985120382: tensor([1, 2, 3, 4, 5])}

In [95]:
x = "asdf"

In [96]:
bob._objects

{71226330779: tensor([1, 2, 3, 4, 5]), 23985120382: tensor([1, 2, 3, 4, 5])}

In [97]:
del x

In [98]:
bob._objects

{71226330779: tensor([1, 2, 3, 4, 5]), 23985120382: tensor([1, 2, 3, 4, 5])}

In [99]:
bob = bob.clear_objects()
bob._objects

{}

In [100]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [101]:
bob._objects

{34969688298: tensor([1, 2, 3, 4, 5])}

In [103]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [104]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually located on another machine (is a PointerTensor). Call .get() on a the PointerTensor or .send({tensor_b.location.id}) on the other tensor.
Tensor A: {tensor_a}
Tensor B: {tensor_b}

In [105]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [106]:
z = x + y

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on {tensor_a.location} while the other is on {tensor_b.location}. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [107]:
from torch import nn, optim

In [108]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [109]:
# A Toy Model
model = nn.Linear(2,1)

In [110]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [111]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(8.5866)
tensor(1.7784)
tensor(0.8621)
tensor(0.5340)
tensor(0.3427)
tensor(0.2212)
tensor(0.1430)
tensor(0.0927)
tensor(0.0602)
tensor(0.0392)
tensor(0.0256)
tensor(0.0168)
tensor(0.0110)
tensor(0.0073)
tensor(0.0048)
tensor(0.0032)
tensor(0.0022)
tensor(0.0015)
tensor(0.0010)
tensor(0.0007)


In [112]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [113]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [114]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [115]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [123]:
train()

tensor(0.0824, requires_grad=True)
tensor(0.0006, requires_grad=True)
tensor(5.0509e-06, requires_grad=True)
tensor(1.8660e-07, requires_grad=True)
tensor(1.2442e-07, requires_grad=True)
tensor(1.0333e-07, requires_grad=True)
tensor(8.5991e-08, requires_grad=True)
tensor(7.1573e-08, requires_grad=True)
tensor(5.9562e-08, requires_grad=True)
tensor(4.9569e-08, requires_grad=True)
tensor(4.1269e-08, requires_grad=True)
tensor(3.4340e-08, requires_grad=True)
tensor(2.8568e-08, requires_grad=True)
tensor(2.3789e-08, requires_grad=True)
tensor(1.9802e-08, requires_grad=True)
tensor(1.6471e-08, requires_grad=True)
tensor(1.3715e-08, requires_grad=True)
tensor(1.1405e-08, requires_grad=True)
tensor(9.4892e-09, requires_grad=True)
tensor(7.9073e-09, requires_grad=True)
tensor(6.5776e-09, requires_grad=True)
tensor(5.4857e-09, requires_grad=True)
tensor(4.5577e-09, requires_grad=True)
tensor(3.8027e-09, requires_grad=True)
tensor(3.1617e-09, requires_grad=True)
tensor(2.6215e-09, requires_grad=

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [116]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

<VirtualWorker id:alice #tensors:0>

In [117]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [118]:
x = x.send(alice)

In [119]:
bob._objects

{47167214565: tensor([1, 2, 3, 4, 5])}

In [120]:
alice._objects

{77184708872: (Wrapper)>[PointerTensor | alice:77184708872 -> bob:47167214565]}

In [121]:
y = x + x

In [122]:
y

(Wrapper)>[PointerTensor | me:33651809309 -> alice:72962389046]

In [123]:
bob._objects

{47167214565: tensor([1, 2, 3, 4, 5]),
 45730011059: tensor([ 2,  4,  6,  8, 10])}

In [124]:
alice._objects

{77184708872: (Wrapper)>[PointerTensor | alice:77184708872 -> bob:47167214565],
 72962389046: (Wrapper)>[PointerTensor | alice:72962389046 -> bob:45730011059]}

In [125]:
jon = sy.VirtualWorker(hook, id="jon")

In [126]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [127]:
bob._objects

{38163454743: tensor([1, 2, 3, 4, 5])}

In [128]:
alice._objects

{6437841308: (Wrapper)>[PointerTensor | alice:6437841308 -> bob:38163454743]}

In [129]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:6437841308 -> bob:38163454743]

In [130]:
bob._objects

{38163454743: tensor([1, 2, 3, 4, 5])}

In [131]:
alice._objects

{}

In [132]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [133]:
bob._objects

{}

In [134]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [135]:
bob._objects

{60827508473: tensor([1, 2, 3, 4, 5])}

In [136]:
alice._objects

{17188052671: (Wrapper)>[PointerTensor | alice:17188052671 -> bob:60827508473]}

In [137]:
del x

In [138]:
bob._objects

{}

In [139]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [140]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [141]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [142]:
bob._objects

{60215279723: tensor([1, 2, 3, 4, 5])}

In [143]:
alice._objects

{}

In [144]:
x.move(alice)

(Wrapper)>[PointerTensor | me:55438114105 -> alice:60215279723]

In [145]:
bob._objects

{}

In [146]:
alice._objects

{60215279723: tensor([1, 2, 3, 4, 5])}

In [147]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [148]:
bob._objects

{54981746604: tensor([1, 2, 3, 4, 5])}

In [149]:
alice._objects

{60215279723: tensor([1, 2, 3, 4, 5]),
 53271622125: (Wrapper)>[PointerTensor | alice:53271622125 -> bob:54981746604]}

In [150]:
x.remote_get()

(Wrapper)>[PointerTensor | me:50112892819 -> alice:53271622125]

In [151]:
bob._objects

{}

In [152]:
alice._objects

{60215279723: tensor([1, 2, 3, 4, 5]), 53271622125: tensor([1, 2, 3, 4, 5])}

In [153]:
x.move(bob)

(Wrapper)>[PointerTensor | me:50112892819 -> bob:53271622125]

In [154]:
x

(Wrapper)>[PointerTensor | me:50112892819 -> bob:53271622125]

In [155]:
bob._objects

{53271622125: tensor([1, 2, 3, 4, 5])}

In [156]:
alice._objects

{60215279723: tensor([1, 2, 3, 4, 5])}

# Section Project:

For the final project for this section, you're going to train on the MNIST dataset using federated learning However the gradient should not come up to central server in raw form

In [157]:
import torchvision.datasets as datasets
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)

UnsupportedNodeError: import statements aren't supported:
  File "C:\ProgramData\Miniconda3\lib\site-packages\syft\generic\frameworks\hook\hook.py", line 2471
    .. include:: cuda_deterministic_backward.rst
    """
    from .modules.utils import _ntuple
    ~~~~ <--- HERE

    def _check_size_scale_factor(dim):
'_onnx_heatmaps_to_keypoints' is being compiled since it was called from '_onnx_heatmaps_to_keypoints_loop'
  File "C:\ProgramData\Miniconda3\lib\site-packages\torchvision\models\detection\roi_heads.py", line 218

    for i in range(int(rois.size(0))):
        xy_preds_i, end_scores_i = _onnx_heatmaps_to_keypoints(maps, maps[i],
        ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~...  <--- HERE
                                                               widths_ceil[i], heights_ceil[i],
                                                               widths[i], heights[i],
